# Inference Pipeline and Format Selection

This notebook demonstrates model format selection and the inference pipeline. Inspired by [Intro to Inference: How to Run AI Models on a GPU](https://developers.google.com/learn/pathways/ai-models-on-gpu-intro).

Run the setup cell first. Use **Runtime > Change runtime type > GPU** in Colab for faster inference.

**Configure runtime first:** Runtime > Change runtime type > Hardware accelerator: **GPU** (T4) > Save.

In [1]:
# Colab setup: clone repo and install dependencies (run this cell first)
try:
    import google.colab
    get_ipython().system("git clone -q https://github.com/KarthikSriramGit/Project-Insight.git")
    get_ipython().run_line_magic("cd", "Project-Insight")
    get_ipython().system("pip install -q -r requirements.txt")
except Exception:
    pass

/content/Project-Insight


In [2]:
# Setup: Colab runs from repo root after clone
import sys
from pathlib import Path
ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print(f"ROOT={ROOT}")

ROOT=/content/Project-Insight


## 1. Format selection by use case

In [3]:
from src.inference.format_selector import select_format

for use_case in ["research", "sharing", "local", "production", "portable"]:
    fmt, rationale = select_format(use_case, hardware="gpu")
    print(f"{use_case}: {fmt}")
    print(f"  {rationale[:85]}...")
    print()

research: safetensors
  Fast, secure weight serialization. Memory-mapped loading, no arbitrary code execution...

sharing: safetensors
  Fast, secure weight serialization. Memory-mapped loading, no arbitrary code execution...

local: gguf
  Compact, quantized format for local inference. Powers llama.cpp and run-on-laptop wor...

production: tensorrt
  Compiled engine for NVIDIA GPUs. Pre-optimized kernels, lowest latency and highest th...

portable: onnx
  Graph-level interchange format. Framework-agnostic, runs on ONNX Runtime, OpenVINO, T...



## 2. Inference metrics (p50, p90, throughput)

In [4]:
from src.inference.metrics import compute_metrics

total_latencies = [1.2, 1.1, 1.3, 1.0, 1.2]
first_token_latencies = [0.1, 0.09, 0.11, 0.1, 0.1]
token_counts = [64, 64, 64, 64, 64]

metrics = compute_metrics(
    total_latencies=total_latencies,
    first_token_latencies=first_token_latencies,
    token_counts=token_counts,
)
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

p50_latency_s: 1.2000
p90_latency_s: 1.2600
p50_ttft_s: 0.1000
p90_ttft_s: 0.1060
throughput_sustained_tok_s: 55.1724


## 3. Inference pipeline with Gemma 2 2B (data-grounded telemetry Q&A)

Uses **Google Gemma 2 2B Instruct** to answer questions about fleet telemetry — the same data from notebook 01. This is the **local** version of the telemetry-to-insight pipeline (notebook 03 scales this to NIM on GKE).

**Setup (one-time):**
1. Accept the license at [google/gemma-2-2b-it](https://huggingface.co/google/gemma-2-2b-it)
2. Create a Read token at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
3. In Colab: click the **key icon** (Secrets) > add `HF_TOKEN` with your token > toggle Notebook access ON

In [ ]:
# Load telemetry data (same as notebooks 01 & 03) — generates synthetic data if missing
import subprocess
from pathlib import Path

data_path = ROOT / "data" / "synthetic" / "fleet_telemetry.parquet"
if not data_path.exists():
    subprocess.run([
        "python", "data/synthetic/generate_telemetry.py",
        "--rows", "100000", "--output-dir", "data/synthetic", "--format", "parquet",
    ], check=True, cwd=str(ROOT))

from src.query.engine import TelemetryQueryEngine

engine = TelemetryQueryEngine(
    data_path=str(data_path),
    nim_base_url="http://localhost:8000",  # not used — we call the local model
    max_context_rows=500,
)
print(f"Telemetry data loaded. Path: {data_path}")

In [5]:
# Inference pipeline with Gemma 2 2B (requires GPU runtime + HF token)
import os
import warnings
warnings.filterwarnings("ignore", message=".*torch_dtype.*")

# Authenticate with HuggingFace (reads HF_TOKEN from Colab Secrets)
try:
    from google.colab import userdata
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    print("HF token loaded from Colab Secrets.")
except Exception:
    if "HF_TOKEN" not in os.environ:
        print("WARNING: Set HF_TOKEN in Colab Secrets (key icon) or as environment variable.")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from src.inference.pipeline import InferencePipeline

model_id = "google/gemma-2-2b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
print(f"Model: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
model = model.to(device)

pipe = InferencePipeline(model, tokenizer, device=device, max_new_tokens=256)

from src.query.prompts import SYSTEM_PROMPT, format_user_query

def clean_response(raw_text, prompt_text=""):
    """Extract only the generated response, remove chat tags, trim to complete sentence."""
    text = raw_text
    if prompt_text and text.startswith(prompt_text):
        text = text[len(prompt_text):]
    for tag in ["<start_of_turn>", "<end_of_turn>", "<bos>", "<eos>", "model\n", "user\n"]:
        text = text.replace(tag, "")
    text = text.strip()
    if text and text[-1] not in ".!?":
        last_end = max(text.rfind("."), text.rfind("!"), text.rfind("?"))
        if last_end > 0:
            text = text[:last_end + 1]
    return text.strip()

# Each query: (question, vehicle_ids, sensor_type, or skip_data=True for general knowledge)
QUERY_CONFIG = [
    {"query": "What was the peak brake pressure percentage in vehicle V001?", "vehicle_ids": ["V001"], "sensor_type": "can"},
    {"query": "Summarize the fleet telemetry health for the vehicles (average speed, throttle, brake usage).", "sensor_type": "can"},
    {"query": "What model format should I use for production inference on NVIDIA GPUs?", "skip_data": True},
]

for cfg in QUERY_CONFIG:
    q = cfg["query"]
    if cfg.get("skip_data"):
        context = "No telemetry data — this is a general knowledge question about ML deployment."
    else:
        df = engine.retrieve(
            vehicle_ids=cfg.get("vehicle_ids"),
            sensor_type=cfg.get("sensor_type"),
        )
        context = engine._data_to_context(df)
    user_msg = f"{SYSTEM_PROMPT}\n\n{format_user_query(q, context)}"
    chat_prompt = f"<start_of_turn>user\n{user_msg}<end_of_turn>\n<start_of_turn>model\n"
    out = pipe.generate([chat_prompt], max_new_tokens=256)
    response = clean_response(out[0], chat_prompt)
    print(f"Q: {q}")
    print(f"A: {response}")
    print("-" * 60)

Device: cuda
Model: microsoft/Phi-3-mini-4k-instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

modeling_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

KeyError: 'type'